In [2]:
print('Importing packages...')

import os
import pandas as pd
import numpy as np
import cufflinks as cf
from bs4 import BeautifulSoup
from IPython.display import HTML
import configparser as cp
cf.set_config_file(offline=True)
import requests
import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import *
import time



import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

print('...Complete')

Importing packages...


...Complete


In [ ]:
# Download csv of ASX200 stocks

print('Downloading ASX200...')
url = 'https://www.asx200list.com/uploads/csv/20190501-asx200.csv'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}

r = requests.get(url, headers=headers)

with open('20190501-asx200.csv', 'wb') as f:
    f.write(r.content)

print('...Complete')

In [ ]:
# Clean csv of ASX200 stocks & export into new iterable csv

print('Cleaning and exporting new csv...')
asx = pd.read_csv('20190501-asx200.csv')

asx = asx.drop(0,axis=0)
asx = asx.drop(asx.columns[1:], axis=1)

asx.to_csv('asx200.csv', index=False, header=None)
print('...Complete')

In [10]:
# Eikon handshake

import eikon as tr
#tr.set_app_key('04851d8cdc3d47f7882782edf7d9abbf5999a94a') #CBS02
tr.set_app_key('c15c88926d37436798d4173b551fe4cebcb3605d') #CBS01

#A valid app key is required.
df = tr.get_news_headlines(query="APT.AX")
print(df)

2020-09-02 16:55:25,704 P[22764] [MainThread 3100] Error on handshake port 9000 : ReadTimeout(ReadTimeout())


                                          versionCreated  \
2020-09-02 05:19:09.000        2020-09-02 07:21:53+00:00   
2020-09-02 02:52:06.000        2020-09-02 02:52:06+00:00   
2020-09-02 02:34:57.000        2020-09-02 02:34:57+00:00   
2020-09-02 00:02:47.798 2020-09-02 00:03:03.460000+00:00   
2020-09-02 00:01:21.533 2020-09-02 00:01:21.533000+00:00   
2020-09-02 00:01:20.369 2020-09-02 00:01:20.369000+00:00   
2020-09-01 23:50:41.626 2020-09-01 23:50:49.354000+00:00   
2020-09-01 23:48:26.260 2020-09-01 23:48:26.260000+00:00   
2020-09-01 23:48:25.111 2020-09-01 23:48:25.111000+00:00   
2020-09-01 07:46:33.420 2020-09-01 07:46:51.259000+00:00   

                                                                      text  \
2020-09-02 05:19:09.000  UPDATE 2-PayPal entry punishes Australia's pri...   
2020-09-02 02:52:06.000  PayPal entry punishes Australia's pricey buy-n...   
2020-09-02 02:34:57.000  BUZZ-The PayPal headache: buy now, pay later s...   
2020-09-02 00:02:47.798    

In [4]:
# Build list of ASX200 stocks with .AX suffixed

file = open('asx200.csv')

stonks = []
for i in file:
    stonks.append(i.rstrip()+'.AX')
print('Complete')

Complete


In [ ]:
try:
    os.mkdir('Articles')
except FileExistsError:
    pass

In [20]:
monthsback = 15
artcount = 0

todaydate = datetime.today() + relativedelta(months=-monthsback)        

cumcalls = 0
for ticker in stonks[0:199]:
    # Build directories for each ticker
    try:
        os.mkdir('Articles/'+ticker[:-3])
    except FileExistsError:
        pass
    
    # Setup query term
    query = "R:" + ticker + " AND Language:LEN" #MODIFY FOR NASDAQ ETC
    num = artcount
    calls = 0
    # Looping through full historical time window (15 months) dated from today. 
    for i in range(monthsback): #15
        datestart = todaydate + relativedelta(months=+i)
        dateend = datestart + relativedelta(months=+1)
        dateto = str(dateend.isoformat()[:-15:])
        
        # Eikon query (API capped to 100 results per query)
        headlines = tr.get_news_headlines(query=query, raw_output=True, 
                                  date_from=datestart.isoformat()[:-7:], date_to=dateto, count = 100)["headlines"]
        calls += 1
        cumcalls += 1
        
        # If query returns more than 100 results, create a new query based upon the last articles creation date. 
        # Note that queries are stored as dictionaries in reverse chronological order, so the end indexes represent the 
        # oldest results. 
        maxcall = False
        repeat = 0
        if len(headlines) == 100:
            maxcall = True
        
        while maxcall == True and repeat <=2: #Change repeat value to change max number of articles collected for a given month
            repeat +=1
            maxcall = False
            newdate = headlines[99]["firstCreated"][:-5]
            newheadlines = tr.get_news_headlines(query=query, raw_output=True, 
                                  date_from=newdate, date_to=datestart.isoformat()[:-7:], count = 100)["headlines"]
            if len(newheadlines) == 100:
                maxcall = True
            calls +=1
            cumcalls += 1
            headlines = headlines + newheadlines
                
        print("\n\nDownloading articles: ",ticker)
        print(datestart.isoformat()[:-16:] + ': ' + str(len(headlines)))
        print("Cumulative Calls: "+str(cumcalls))
        
        # Once a month of ticker data is retrieved for a stock, the headlines are used to call the stories from the eikon API
        # these are then converted into HTML and parsed. No cleaning is conducted on the text. 
        # export format: Article number; ticker; date published; title; text body
        monthnum = 0
        for i in headlines:
            if i["documentType"] == "Story":
                print(str(monthnum)+' ',end='')
                url = i["storyId"]
                try:
                    article = tr.get_news_story(url)
                except tr.EikonError:
                    continue
                calls += 1
                cumcalls += 1
                raw = BeautifulSoup(article, 'html.parser').get_text()

                mainText = raw.replace(';', ' ')
                title = i['text'].replace(';', ' ')
                date = i['firstCreated']
                export = str(num) + ';' +ticker[:-3] + ';' + date + ';' + title + ';' + mainText 
                outfile = open('Articles/'+ticker[:-3]+'/'+ticker[:-3]+'_'+str(num)+'.txt', 'w',  encoding="utf-8")
                outfile.write(export)
                outfile.close()
                num += 1
                monthnum +=1
            else:
                pass
    print("\n** Calls made for "+ticker+": "+str(calls))
    print("** Cumulative calls made: "+str(cumcalls))
    
        
print("Complete")        



2019-06-03: 10
Cumulative Calls: 1
0 1 

2019-07-03: 6
Cumulative Calls: 4
0 1 

2019-08-03: 38
Cumulative Calls: 7
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

2019-09-03: 18
Cumulative Calls: 28
0 1 2 3 4 5 6 7 

2019-10-03: 13
Cumulative Calls: 37
0 1 2 

2019-11-03: 10
Cumulative Calls: 41
0 1 2 3 

2019-12-03: 4
Cumulative Calls: 46
0 1 

2020-01-03: 3
Cumulative Calls: 49
0 1 

2020-02-03: 38
Cumulative Calls: 52
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

2020-03-03: 67
Cumulative Calls: 73
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 

2020-04-03: 59
Cumulative Calls: 113
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 

2020-05-03: 39
Cumulative Calls: 147
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 

2020-06-03: 28
Cumulative Calls: 170
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 

2020-07-03: 11
Cumulative Calls: 190
0 1 2 3 4 5 6 

2020-08-0